# LA County Board of Supervisors
An analysis of donations to 2nd District candidates

### Read libraries

In [1]:
import json
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
from matplotlib import rc
%matplotlib inline
import numpy as np
import geojson
import jenkspy
import altair as alt
alt.renderers.enable('notebook')
import altair as alt
import altair_latimes as lat
alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

### Load full list of donations to county candidates since 2007

In [2]:
contributions = pd.read_csv('input/contributions.csv',low_memory=False, dtype={'contributor_zip_code': 'str', 'contributor_zip_code_ext': 'str'})

contributions.contributor_address_2 = contributions.contributor_address_2.fillna('')
contributions.occupation = contributions.occupation.fillna('')
contributions.employer = contributions.employer.fillna('')
contributions.description = contributions.description.fillna('')
contributions.contributor_first_name = contributions.contributor_first_name.fillna('')
contributions.contributor_zip_code_ext = contributions.contributor_zip_code_ext.fillna('')

In [3]:
contributions.columns

Index(['last_name', 'first_name', 'committee_id', 'committee_name',
       'office_type', 'district_number', 'schedule', 'type', 'period_beg_date',
       'period_end_date', 'election_date', 'date', 'amount_rcvd', 'amount_pd',
       'description', 'contributor_first_name', 'contributor_last_name',
       'contributor_address', 'contributor_address_2', 'contributor_city',
       'contributor_state', 'contributor_zip_code', 'contributor_zip_code_ext',
       'occupation', 'employer', 'rec_id', 'cycle'],
      dtype='object')

### Clean up the table

In [4]:
#names
contributions['amount_rcvd'] = contributions['amount_rcvd'].astype(int)
contributions['monthyear'] = pd.to_datetime(contributions['date']).map(lambda dt: dt.replace(day=1))
contributions['type'] = contributions['type'].str.replace("Small Contributor Committee", "Small Donor Committee")
contributions['first_name'] = contributions['first_name'].str.upper()
contributions['last_name'] = contributions['last_name'].str.upper()
contributions['occupation'] = contributions['occupation'].str.upper()
contributions['employer'] = contributions['employer'].str.upper()
contributions['committee_name'] = contributions['committee_name'].str.upper()
contributions['candidate_name'] =  contributions["first_name"].str.title() + ' ' + contributions["last_name"].str.title()
contributions['candidate_name'] = contributions['candidate_name'].str.replace('Holly J. Mitchell', 'Holly Mitchell')
contributions['first_name'] = contributions['first_name'].str.replace('HOLLY J.', 'HOLLY')
contributions["nu_contributor_address"] = contributions["contributor_address"].str.split('#').str[0].str.split(', ').str[0].str.split('Ste').str[0].str.split('Suite').str[0].str.split(',').str[0]
contributions['full_address'] =  contributions["nu_contributor_address"] + ', ' + contributions["contributor_city"] + ', ' + contributions["contributor_state"] + ', ' + contributions["contributor_zip_code"].astype(str)
contributions['candidate_slug'] =  contributions["first_name"].str.lower() + '_' + contributions["last_name"].str.lower()
contributions['candidate_slug'] = contributions['candidate_slug'].str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('.', '').str.replace('\'', '')

#time
contributions['date'] = pd.to_datetime(contributions['date'])
contributions['year'], contributions['month'] = contributions['date'].dt.year, contributions['date'].dt.month
contributions['year'] = contributions['year'].astype(str)
contributions['month'] = contributions['month'].astype(str)
contributions['months'] = contributions['date'].dt.strftime('%m/%Y').astype(str)
contributions['week'] = contributions['date'].dt.week
contributions['week'] = contributions['week'].astype(str)

### Filter data for recent contributions and money in the District 2 race

In [5]:
recent_contributions = contributions[(contributions['date'] > '2018-06-30')]
second_district_candidates = (['Holly Mitchell','Herb Wesson','Jan Perry','Chan Jeong'])
districttwo = recent_contributions[recent_contributions['candidate_name'].isin(second_district_candidates)]

herb = (['Herb Wesson'])
herbsupervisor = recent_contributions[recent_contributions['candidate_name'].isin(herb)]

In [6]:
districttwo[['candidate_name','contributor_first_name', 'contributor_last_name','occupation', 'employer','amount_rcvd']]

,candidate_name,contributor_first_name,contributor_last_name,occupation,employer,amount_rcvd
1541,Herb Wesson,,UFCW Western States Council,,,1500
1542,Herb Wesson,,UFCW Local 324 PAC,,,1500
1543,Herb Wesson,,Angelenos for Safe Transportation PAC Sponsore...,,,1500
1544,Herb Wesson,,United Food and Commercial Workers Internation...,,,1500
1545,Herb Wesson,,United Food and Commercial Workers Union Local...,,,1500
1546,Herb Wesson,,"UFCW Local 1167, AFL-CIO-CLC",,,1500
1547,Herb Wesson,,Autumn Burke for Assembly 2018,,,1500
1548,Herb Wesson,,Janna Zurita for City Council 2019,,,1500
1549,Herb Wesson,,Hertzberg for Senate 2018,,,1500
1550,Herb Wesson,,Steven Bradford for Senate 2020,,,1500


In [7]:
districttwo_indiv = districttwo[districttwo.type == 'I']

In [8]:
districttwo_indiv

,last_name,first_name,committee_id,committee_name,office_type,district_number,schedule,type,period_beg_date,period_end_date,...,cycle,monthyear,candidate_name,nu_contributor_address,full_address,candidate_slug,year,month,months,week
59499,WESSON,HERB,1414475,WESSON FOR SUPERVISOR 2020,CSU,NaN,A,I,1/1/18,12/31/18,...,2018-2019,2018-12-01,Herb Wesson,5514 Overdale Dr,"5514 Overdale Dr, Los Angeles, CA, 90043",herb_wesson,2018,12,12/2018,49
59500,WESSON,HERB,1414475,WESSON FOR SUPERVISOR 2020,CSU,NaN,A,I,1/1/18,12/31/18,...,2018-2019,2018-12-01,Herb Wesson,800 W 1st St,"800 W 1st St, Los Angeles, CA, 90012",herb_wesson,2018,12,12/2018,49
59501,WESSON,HERB,1414475,WESSON FOR SUPERVISOR 2020,CSU,NaN,A,I,1/1/18,12/31/18,...,2018-2019,2018-12-01,Herb Wesson,1570 Prospect Ave,"1570 Prospect Ave, Hermosa Beach, CA, 90254",herb_wesson,2018,12,12/2018,49
59502,WESSON,HERB,1414475,WESSON FOR SUPERVISOR 2020,CSU,NaN,A,I,1/1/18,12/31/18,...,2018-2019,2018-12-01,Herb Wesson,610 S Main St,"610 S Main St, Los Angeles, CA, 90014",herb_wesson,2018,12,12/2018,49
59503,WESSON,HERB,1414475,WESSON FOR SUPERVISOR 2020,CSU,NaN,A,I,1/1/18,12/31/18,...,2018-2019,2018-12-01,Herb Wesson,12100 Wilshire Blvd,"12100 Wilshire Blvd, Los Angeles, CA, 90025",herb_wesson,2018,12,12/2018,49
59504,WESSON,HERB,1414475,WESSON FOR SUPERVISOR 2020,CSU,NaN,A,I,1/1/18,12/31/18,...,2018-2019,2018-12-01,Herb Wesson,6002 Beaumont Ave,"6002 Beaumont Ave, La Jolla, CA, 92037",herb_wesson,2018,12,12/2018,49
59505,WESSON,HERB,1414475,WESSON FOR SUPERVISOR 2020,CSU,NaN,A,I,1/1/18,12/31/18,...,2018-2019,2018-12-01,Herb Wesson,584 Rosemont Ave,"584 Rosemont Ave, Pasadena, CA, 91103",herb_wesson,2018,12,12/2018,49
59506,WESSON,HERB,1414475,WESSON FOR SUPERVISOR 2020,CSU,NaN,A,I,1/1/18,12/31/18,...,2018-2019,2018-12-01,Herb Wesson,255 S Grand Ave,"255 S Grand Ave, Los Angeles, CA, 90012",herb_wesson,2018,12,12/2018,49
59507,WESSON,HERB,1414475,WESSON FOR SUPERVISOR 2020,CSU,NaN,A,I,1/1/18,12/31/18,...,2018-2019,2018-12-01,Herb Wesson,15477 Ventura Blvd,"15477 Ventura Blvd, Sherman Oaks, CA, 91403",herb_wesson,2018,12,12/2018,49
59508,WESSON,HERB,1414475,WESSON FOR SUPERVISOR 2020,CSU,NaN,A,I,1/1/18,12/31/18,...,2018-2019,2018-12-01,Herb Wesson,2747 Winter St,"2747 Winter St, Los Angeles, CA, 90033",herb_wesson,2018,12,12/2018,49


### How much did each occupation give?

In [9]:
second_district_occupations = districttwo_indiv.groupby(['occupation'])['amount_rcvd'].sum().reset_index().sort_values(by='occupation', ascending=True)

In [10]:
# second_candidates_occupations_pivot = pd.pivot_table(second_district_occupations,values='amount_rcvd', index=['occupation'], columns=['candidate_name'], aggfunc=[np.sum], fill_value='0').reset_index()

In [11]:
second_district_occupations

,occupation,amount_rcvd
0,,4500
1,ACCOUNTANT,10300
2,ACCOUNTANT / LICENSED PROCESS SERVER,800
3,ACCOUNTING,1500
4,ACTOR,2950
5,ACTOR-DIRECTOR,1500
6,ACTRESS,2500
7,ADJUNCT PROFESSOR,300
8,ADMINISTRATION,700
9,ADMINISTRATIVE ASSISTANT,1600


In [12]:
contributions.columns

Index(['last_name', 'first_name', 'committee_id', 'committee_name',
       'office_type', 'district_number', 'schedule', 'type', 'period_beg_date',
       'period_end_date', 'election_date', 'date', 'amount_rcvd', 'amount_pd',
       'description', 'contributor_first_name', 'contributor_last_name',
       'contributor_address', 'contributor_address_2', 'contributor_city',
       'contributor_state', 'contributor_zip_code', 'contributor_zip_code_ext',
       'occupation', 'employer', 'rec_id', 'cycle', 'monthyear',
       'candidate_name', 'nu_contributor_address', 'full_address',
       'candidate_slug', 'year', 'month', 'months', 'week'],
      dtype='object')

### Specific donor lookup

In [13]:
contributions[contributions['contributor_last_name'].str.contains('Juul')]

,last_name,first_name,committee_id,committee_name,office_type,district_number,schedule,type,period_beg_date,period_end_date,...,cycle,monthyear,candidate_name,nu_contributor_address,full_address,candidate_slug,year,month,months,week
76399,WESSON,HERB,1414475,WESSON FOR SUPERVISOR 2020,CSU,NaN,A,O,1/1/18,12/31/18,...,2018-2019,2018-12-01,Herb Wesson,560 20th St,"560 20th St, San Francisco, CA, 94107",herb_wesson,2018,12,12/2018,52


In [15]:
herbsupervisor.to_csv('output/wesson-district2-2018-19')